In [1]:
import random
import matplotlib
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [8]:
#kmeans
class KMeans:
    def __init__(self, k):
        self.k= k
        self.means=None
        
    def classify(self, input):
        return min(range(self.k), key = lambda i: squared_distance(input, self.means[i]))
    
    def train(self, inputs):
        self.means = random.sample(inputs, self.k)
        assignments = None
        while True:
            new_assignments = map(self.classify, inputs)
            if assignments == new_assignments:
                return
            assignments = new_assignements
            for i in range(self.k):
                i_points = [p for p, a in zip(inputs, assignments)if a ==1]
                
                if i_points:
                    self.means[i] = vector_mean(i_points)
            

In [9]:
random.seed(0)
clusterer=KMeans(3)
clusterer.train(inputs)
print clusterer.means

NameError: name 'inputs' is not defined

In [7]:
def vector_add(v, w):
    return [v_i+w_i for v_i, w_i in zip(v,w)]
def vector_sum(vectors):
    return reduce(vector_add, vectors)
def scalar_multiply(c,v):
    return [c*v_i for v_i in v]
def vector_mean(vectors):
    n=len(vectors)
    return scalar_multiply(1/n, vector_sum(vectors))
